In [3]:
from google.cloud import language_v1

def sample_analyze_entity_sentiment(text_content):
    """
    Analyzing Entity Sentiment in a String with Custom Categories

    Args:
      text_content: The text content to analyze
    """

    client = language_v1.LanguageServiceClient()

    # Set document type to PLAIN_TEXT
    type_ = language_v1.types.Document.Type.PLAIN_TEXT

    # Optionally specify the language, or let it be automatically detected.
    language = "en"
    document = {"content": text_content, "type_": type_, "language": language}

    # Set the encoding type to UTF8
    encoding_type = language_v1.EncodingType.UTF8

    # Perform entity sentiment analysis
    response = client.analyze_entity_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )

    # Process each entity in the response
    for entity in response.entities:
        print(f"Representative name for the entity: {entity.name}")
        print("Entity type: {}".format(language_v1.Entity.Type(entity.type_).name))
        print("Salience score: {:.2f}".format(entity.salience))

        # Get the sentiment score and categorize it
        sentiment_score = entity.sentiment.score
        if 0.0 <= sentiment_score <= 1.0:
            sentiment = 'Frustrated'
        elif 1.1 <= sentiment_score <= 2.5:
            sentiment = 'Unsatisfied'
        elif 2.6 <= sentiment_score <= 3.5:
            sentiment = 'Neutral'
        elif 3.6 <= sentiment_score <= 4.5:
            sentiment = 'Satisfied'
        elif 4.6 <= sentiment_score <= 5.0:
            sentiment = 'Excited'
        print("Entity sentiment score: {:.2f}".format(sentiment_score))
        print("Entity sentiment category: {}".format(sentiment))

        # Print additional entity details
        for metadata_name, metadata_value in entity.metadata.items():
            print(f"{metadata_name} = {metadata_value}")

        for mention in entity.mentions:
            print(f"Mention text: {mention.text.content}")
            print("Mention type: {}".format(language_v1.EntityMention.Type(mention.type_).name))

    # Print the detected language of the text
    print("Language of the text: {}".format(response.language))

# Example text to analyze
text_content = 'Please elaborate on why you have selected DBS home loan.Please do not provide any sensitive personal information, including login passwords or one-time passwords.:Very easy to apply and complete all documents and send to DBS WhatsApp banker they will check first before submitting.'
sample_analyze_entity_sentiment(text_content)


Representative name for the entity: home
Entity type: LOCATION
Salience score: 0.22
Entity sentiment score: 0.10
Entity sentiment category: Frustrated
Mention text: home
Mention type: COMMON
Representative name for the entity: passwords.
Entity type: OTHER
Salience score: 0.16
Entity sentiment score: 0.10
Entity sentiment category: Frustrated
Mention text: passwords.
Mention type: COMMON
Representative name for the entity: DBS
Entity type: ORGANIZATION
Salience score: 0.15
Entity sentiment score: 0.10
Entity sentiment category: Frustrated
wikipedia_url = https://en.wikipedia.org/wiki/DBS_Bank
mid = /m/01q7n5
Mention text: DBS
Mention type: PROPER
Representative name for the entity: login passwords
Entity type: OTHER
Salience score: 0.12
Entity sentiment score: 0.10
Entity sentiment category: Frustrated
Mention text: login passwords
Mention type: COMMON
Representative name for the entity: information
Entity type: OTHER
Salience score: 0.12
Entity sentiment score: 0.10
Entity sentiment c

In [10]:
from google.cloud import language_v1

def sample_analyze_entity_sentiment(text_content):
    """
    Analyzing Entity Sentiment in a String with Custom Categories and Specific Entities

    Args:
      text_content: The text content to analyze
    """

    client = language_v1.LanguageServiceClient()

    # Set document type to PLAIN_TEXT
    type_ = language_v1.types.Document.Type.PLAIN_TEXT

    # Optionally specify the language, or let it be automatically detected.
    language = "en"
    document = {"content": text_content, "type_": type_, "language": language}

    # Set the encoding type to UTF8
    encoding_type = language_v1.EncodingType.UTF8

    # List of specific entities to focus on, case insensitive
    target_entities = [
        "debit cards", "credit cards", "personal loan", "cashline", "education loan", "tuition fee loan",
        "renovation loan", "mortgage loan", "home loan", "car loan", "auto loan", "digibank app", "internet banking", "paylah!", "vickers",
        "unit-trust", "non-unit trust", "equities", "digiportfolio", "treasures relationship manager", "ssb", 'atm',
        "vtm", "phone banking", "coin deposit machine", "general insurance", "life insurance", "payments",
        "dbs deposit account", "paynow", "cheque", "giro", "digivault", "dbs hotline", "dbs branches", "staff",
        "overseas transfer", "contact center", "others", "dbs wealth planning manager"
    ]

    # Perform entity sentiment analysis
    response = client.analyze_entity_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )

    # Process each entity in the response
    for entity in response.entities:
        entity_name_lower = entity.name.lower()
        if any(target in entity_name_lower for target in target_entities):
            print(f"Representative name for the entity: {entity.name}")
            print("Entity type: {}".format(language_v1.Entity.Type(entity.type_).name))
            print("Salience score: {:.2f}".format(entity.salience))

            # Get the sentiment score and categorize it based on score and magnitude
            score = entity.sentiment.score
            magnitude = entity.sentiment.magnitude

            if score > 0.25:
                sentiment = 'Excited' if magnitude > 2 else 'Satisfied'
            elif score < -0.25:
                sentiment = 'Frustrated' if magnitude > 2 else 'Unsatisfied'
            else:
                sentiment = 'Neutral' if magnitude < 1 else 'Mixed'

            print("Entity sentiment score: {:.2f}".format(score))
            print("Entity sentiment magnitude: {:.2f}".format(magnitude))
            print("Entity sentiment category: {}".format(sentiment))

    # Print the detected language of the text
    print("Language of the text: {}".format(response.language))

# Example text to analyze
text_content = 'Please elaborate on why you have selected DBS home loan. Very easy to apply and complete all documents and send to DBS WhatsApp banker they will check first before submitting.'
sample_analyze_entity_sentiment(text_content)


Representative name for the entity: home loan
Entity type: OTHER
Salience score: 0.30
Entity sentiment score: -0.10
Entity sentiment magnitude: 0.10
Entity sentiment category: Neutral
Language of the text: en


In [19]:
from google.cloud import language_v1
import six

def sample_analyze_sentiment(content):
    client = language_v1.LanguageServiceClient()

    # Check if the content contains a colon and split it if it does
    # if ':' in content:
    #     parts = content.split(':', 1)  # Split at the first colon
    #     if len(parts) > 1:
    #         content = parts[1].strip()  # Use only the part after the colon

    # Ensure the content is in UTF-8 if it's in binary type
    if isinstance(content, six.binary_type):
        content = content.decode("utf-8")

    # Define the type of document
    type_ = language_v1.Document.Type.PLAIN_TEXT
    document = {"type_": type_, "content": content}

    # Analyze the sentiment of the document
    response = client.analyze_sentiment(request={"document": document})
    sentiment = response.document_sentiment

    # Print raw score and magnitude
    print("Score: {:.2f}".format(sentiment.score))
    print("Magnitude: {:.2f}".format(sentiment.magnitude))

    # Categorize the sentiment based on score and magnitude
    score = sentiment.score
    magnitude = sentiment.magnitude

    if score >= 0.2:
        sentiment_category = 'Excited' if magnitude > 2 else 'Satisfied'
    elif score <= -0.2:
        sentiment_category = 'Frustrated' if magnitude > 2 else 'Unsatisfied'
    else:
        sentiment_category = 'Neutral' if magnitude < 1.2 else 'Mixed' 
    # Print the categorized sentiment
    print("Categorized Sentiment: {}".format(sentiment_category))

# Example usage
text_content = '''(Optional) We have now come to the end of the survey. Before you go, please share your feedback (if any) for us to improve your experience with [Field-Q1_PipedText][Field-Franchise].(Please do not provide any sensitive personal information, such as NRIC / Passport no., contact no. or login passwords.) Whenever I use the credit card, I  don't have a breakdown, whereas Thrust Card has. I believe it would be beneficial to check items that I used to pay for when using the credit card.'''
sample_analyze_sentiment(text_content)


Score: 0.00
Magnitude: 2.00
Categorized Sentiment: Mixed


In [18]:
import spacy
from google.cloud import language_v1
import six

# Load SpaCy's English language model
nlp = spacy.load("en_core_web_sm")

def sample_analyze_sentiment(content):
    client = language_v1.LanguageServiceClient()

    # Ensure the content is in UTF-8 if it's in binary type
    if isinstance(content, six.binary_type):
        content = content.decode("utf-8")

    # Use SpaCy to parse the content
    doc = nlp(content)

    # Finding comparative structures
    comparative_parts = []
    for token in doc:
        # Check for comparative conjunctions or other relations
        if token.dep_ in ["ccomp", "prep", "xcomp"]:
            ancestor = token.head.text
            subtree_span = doc[token.left_edge.i: token.right_edge.i + 1]
            comparative_parts.append(subtree_span.text)

    if not comparative_parts:
        comparative_parts.append(content)  # Use entire content if no comparative structure found

    # Analyze sentiment for each detected part
    for part in comparative_parts:
        document = {"type_": language_v1.Document.Type.PLAIN_TEXT, "content": part}
        response = client.analyze_sentiment(request={"document": document})
        sentiment = response.document_sentiment
        print(f"Text Part: {part}")
        print("Score: {:.2f}".format(sentiment.score))
        print("Magnitude: {:.2f}".format(sentiment.magnitude))
        # Categorization logic can be applied here as before

        score = sentiment.score
        magnitude = sentiment.magnitude

        if score >= 0.2:
            sentiment_category = 'Excited' if magnitude > 2 else 'Satisfied'
        elif score <= -0.2:
            sentiment_category = 'Frustrated' if magnitude > 2 else 'Unsatisfied'
        else:
            sentiment_category = 'Neutral' if magnitude < 1.2 else 'Mixed' 
        # Print the categorized sentiment
        print("Categorized Sentiment: {}".format(sentiment_category))

# Example usage
text_content = "Whenever I use the credit card, I don't have a breakdown, whereas Thrust Card has. I believe it would be beneficial to check items that I used to pay for when using the credit card."
sample_analyze_sentiment(text_content)


Text Part: it would be beneficial to check items that I used to pay for when using the credit card
Score: 0.00
Magnitude: 0.00
Categorized Sentiment: Neutral
Text Part: to check items that I used to pay for when using the credit card
Score: -0.10
Magnitude: 0.10
Categorized Sentiment: Neutral
Text Part: that I used to pay for when using the credit card
Score: -0.20
Magnitude: 0.20
Categorized Sentiment: Unsatisfied
Text Part: for when using the credit card
Score: 0.10
Magnitude: 0.10
Categorized Sentiment: Neutral


In [2]:
#!/usr/bin/env python
# coding: utf-8

import vertexai
from vertexai.generative_models import GenerativeModel
import vertexai.preview.generative_models as generative_models
import json

def classify_sentiment(text):
    vertexai.init(project="jbaaam", location="us-central1")

    # Define the system instruction text for sentiment analysis
    system_instruction= """
You are a Sentiment Analyzer for a Bank. Analyze customer feedback texts to extract meaningful insights and sentiment. Follow these specific instructions to process the text:

<INSTRUCTIONS>
 1. If the feedback includes a \"question:answer\" format (separated by the first colon in the text), use the question as context for understanding the nature of the feedback for example if it is comparative in nature.
 2. Identify key comparative words or phrases that indicate a comparison with other banks or products.
 3. Generate a sentiment score on a scale from 0.0 to 5.0, where 0 indicates extreme dissatisfaction and 5 indicates high satisfaction.
 4. Categorize the sentiment based on the following ranges:
    - 0.0 to 1.0: \'Frustrated\': Indicates severe dissatisfaction from unresolved or recurring issues. Customers are unlikely to recommend the product or service and may warn others against using it.

    - 1.1 to 2.5: \'Unsatisfied\': Reflects displeasure from specific shortcomings. Customers are hesitant to recommend and may highlight negative aspects when discussing the product or service.

    - 2.6 to 3.5: \'Neutral\': Represents indifference; experiences are neither significantly positive nor negative. Recommendations may be lukewarm or noncommittal. Short feedback with no/minimal context

    - 3.6 to 4.5: \'Satisfied\': Customers are generally pleased and feel their expectations have been met. They are likely to recommend the product or service, acknowledging some minor flaws.

    - 4.6 to 5.0: \'Excited\': Shows high enthusiasm and exceeded expectations. Customers actively promote the product or service, sharing positive experiences and recommending it highly.5. Provide a detailed description of the sentiment.
 6. Ensure the output is formatted in a valid JSON with sentiment_score, sentiment_category, sentiment_description only
</INSTRUCTIONS>

<EXAMPLES>
<Input>
"Why are you MORE satisfied with DBS' local payment and transfer services than [Field-Competitor_Bank]\'s?: More ATMs, very easy to use (payment/transfer)\"
</Input>
<Output>
{"sentiment_score": .3.7,"sentiment_category": "Satisfied","sentiment_description": "The customer expresses a higher satisfaction with DBS due to more ATMs and easier usability compared to [Field-Competitor_Bank]."}
</Output>

<Input>
"Why are you LESS satisfied with DBS' local payment and transfer services than [Field-Competitor_Bank]\'s?: Credit card rebate payment without tie to monthly spending (payment/transfer)"
</Input>
<Output>
{"sentiment_score": 2.4,"sentiment_category": "Unsatisfied","sentiment_description": "The customer is less satisfied due to the lack of tied rebates with DBS compared to [Field-Competitor_Bank], suggesting a desire for better benefits."}
</Output>

<Input>
"Uninstall paylah when after reinstall have difficulty authentication my email when I tried many times"
</Input>
<Output>
{"sentiment_score":1.3,"sentiment_category":"Frustrated","sentiment_description":"The customer expresses significant frustration due to difficulties in re-authenticating their email after reinstalling the Paylah app, despite multiple attempts. This experience has likely led to a negative perception of the app\'s usability and customer support."}
</Output>

<Input>
"Meh"
</Input>

<Output>
{"sentiment_score":2.6,"sentiment_category":"Neutral","sentiment_description":"The customer has no comments and hence is neutral."}
</Output>


</EXAMPLES>
"""

    # Configure the model
    model = GenerativeModel(
        "gemini-1.5-flash-001",
        system_instruction=[system_instruction]
    )

    # Setup generation configuration
    generation_config = {
        "max_output_tokens": 256,
        "temperature": 0.2,
        "top_p": 0.95,
    }

    # Define safety settings
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [text],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    
    retries = 0
    while retries < max_retries:
        try:
            responses = model.generate_content(
                [text],
                generation_config=generation_config,
                safety_settings=safety_settings,
                stream=False,
            )
            json_data = responses.text
            clean_json_string = json_data.strip('```json \n')
            data = json.loads(clean_json_string)

            return data["sentiment_score"], data["sentiment_category"]
        
        except ResourceExhausted as e:
            logging.error(f"Resource exhausted: {e}. Retrying in 10 seconds...")
            retries += 1
            time.sleep(10)
        
        except json.JSONDecodeError as e:
            logging.error(f"JSON decode error: {e}")
            return None, "Error"
        
        except Exception as e:
            logging.error(f"Error: {e}")
            return None, "Error"
    
    logging.error("Max retries exceeded.")



    # for response in responses:
    #     print(response.text, end="")

   

# Example usage
text_input = """Please elaborate on why you have selected [QID4-ChoiceGroup-SelectedChoices].Please do not provide any sensitive personal information, including login passwords or one-time passwords: Exact amount & monthly rates. (investment digiPortfolio)"""
response_output = classify_sentiment(text_input)
print(response_output)


0
(2.6, 'Neutral')


In [3]:
#!/usr/bin/env python
# coding: utf-8

import vertexai
from vertexai.generative_models import GenerativeModel
import vertexai.preview.generative_models as generative_models
import json
import logging
from google.api_core.exceptions import ResourceExhausted
import time

def classify_sentiment(text):
    vertexai.init(project="jbaaam", location="us-central1")

    # Define the system instruction text for sentiment analysis
    system_instruction= """
You are a Sentiment Analyzer for a Bank. Analyze customer feedback texts to extract meaningful insights and sentiment. Follow these specific instructions to process the text:

<INSTRUCTIONS>
 1. If the feedback includes a \"question:answer\" format (separated by the first colon in the text), use the question as context for understanding the nature of the feedback for example if it is comparative in nature.
 2. Identify key comparative words or phrases that indicate a comparison with other banks or products.
 3. Generate a sentiment score on a scale from 0.0 to 5.0, where 0 indicates extreme dissatisfaction and 5 indicates high satisfaction.
 4. Categorize the sentiment based on the following ranges:
    - 0.0 to 1.0: \'Frustrated\': Indicates severe dissatisfaction from unresolved or recurring issues. Customers are unlikely to recommend the product or service and may warn others against using it.

    - 1.1 to 2.5: \'Unsatisfied\': Reflects displeasure from specific shortcomings. Customers are hesitant to recommend and may highlight negative aspects when discussing the product or service.

    - 2.6 to 3.5: \'Neutral\': Represents indifference; experiences are neither significantly positive nor negative. Recommendations may be lukewarm or noncommittal. Short feedback with no/minimal context

    - 3.6 to 4.5: \'Satisfied\': Customers are generally pleased and feel their expectations have been met. They are likely to recommend the product or service, acknowledging some minor flaws.

    - 4.6 to 5.0: \'Excited\': Shows high enthusiasm and exceeded expectations. Customers actively promote the product or service, sharing positive experiences and recommending it highly.5. Provide a detailed description of the sentiment.
 6. Ensure the output is formatted in a valid JSON with sentiment_score, sentiment_category, sentiment_description only
</INSTRUCTIONS>

<EXAMPLES>
<Input>
"Why are you MORE satisfied with DBS' local payment and transfer services than [Field-Competitor_Bank]\'s?: More ATMs, very easy to use (payment/transfer)\"
</Input>
<Output>
{"sentiment_score": .3.7,"sentiment_category": "Satisfied","sentiment_description": "The customer expresses a higher satisfaction with DBS due to more ATMs and easier usability compared to [Field-Competitor_Bank]."}
</Output>

<Input>
"Why are you LESS satisfied with DBS' local payment and transfer services than [Field-Competitor_Bank]\'s?: Credit card rebate payment without tie to monthly spending (payment/transfer)"
</Input>
<Output>
{"sentiment_score": 2.4,"sentiment_category": "Unsatisfied","sentiment_description": "The customer is less satisfied due to the lack of tied rebates with DBS compared to [Field-Competitor_Bank], suggesting a desire for better benefits."}
</Output>

<Input>
"Uninstall paylah when after reinstall have difficulty authentication my email when I tried many times"
</Input>
<Output>
{"sentiment_score":1.3,"sentiment_category":"Frustrated","sentiment_description":"The customer expresses significant frustration due to difficulties in re-authenticating their email after reinstalling the Paylah app, despite multiple attempts. This experience has likely led to a negative perception of the app\'s usability and customer support."}
</Output>

<Input>
"Meh"
</Input>

<Output>
{"sentiment_score":2.6,"sentiment_category":"Neutral","sentiment_description":"The customer has no comments and hence is neutral."}
</Output>

<Input>
"Online forum discussion about account features"
</Input>
<Output>
{"sentiment_score":2.6,"sentiment_category":"Neutral","sentiment_description":"The feedback does not mention what is the sentiment or opinion but it is just stating, hence it is just neutral."}
</Output>


</EXAMPLES>
"""

    # Configure the model
    model = GenerativeModel(
        "gemini-1.5-flash-001",
        system_instruction=[system_instruction]
    )

    # Setup generation configuration
    generation_config = {
        "max_output_tokens": 256,
        "temperature": 0.2,
        "top_p": 0.95,
    }

    # Define safety settings
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [text],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    
    # retries = 0
    # max_retries=3
    # while retries < max_retries:
    #     try:
    #         responses = model.generate_content(
    #             [text],
    #             generation_config=generation_config,
    #             safety_settings=safety_settings,
    #             stream=False,
    #         )
    #         json_data = responses.text
    #         clean_json_string = json_data.strip('```json \n')
    #         data = json.loads(clean_json_string)
    #         print (retries)

    #         return data["sentiment_score"], data["sentiment_category"]
        
    #     except ResourceExhausted as e:
    #         logging.error(f"Resource exhausted: {e}. Retrying in 10 seconds...")
    #         retries += 1
    #         time.sleep(10)
        
    #     except json.JSONDecodeError as e:
    #         logging.error(f"JSON decode error: {e}")
    #         return None, "Error"
        
    #     except Exception as e:
    #         logging.error(f"Error: {e}")
    #         return None, "Error"
    
    # logging.error("Max retries exceeded.")

    return responses.text

    # for response in responses:
    #     print(response.text, end="")

   

# Example usage
text_input = """Online forum discussion about account features related matter and comparison with other banks"""
response_output = classify_sentiment(text_input)
print(response_output)


Please provide the actual text of the online forum discussion. I need the specific content to analyze the sentiment, identify comparative words, and generate the JSON output as per the instructions. 



In [1]:
###PRO MODELL
#!/usr/bin/env python
# coding: utf-8

import vertexai
from vertexai.generative_models import GenerativeModel
import vertexai.preview.generative_models as generative_models
import json

def classify_sentiment(text):
    vertexai.init(project="jbaaam", location="us-central1")

    # Define the system instruction text for sentiment analysis
    system_instruction= """\"\"\"
You are a Sentiment Analyzer for a Bank. Analyze customer feedback texts to extract meaningful insights and sentiment. Ensure the output is ONLY a valid JSON format with sentiment_score, sentiment_category, sentiment_description only. Follow these specific instructions to process the text:

<INSTRUCTIONS>
1. If the feedback includes a \"question:answer\" format (separated by the first colon in the text), use the question as context for understanding the nature of the feedback for example if it is comparative in nature.
2. Identify key comparative words or phrases that indicate a comparison with other banks or products.
3. Generate a sentiment score on a scale from 0.0 to 5.0, where 0 indicates extreme dissatisfaction and 5 indicates high satisfaction.
4. Categorize the sentiment based on the following ranges:
- 0.0 to 1.0: \'Frustrated\': Indicates severe dissatisfaction from unresolved or recurring issues. Customers are unlikely to recommend the product or service and may warn others against using it.

- 1.1 to 2.5: \'Unsatisfied\': Reflects displeasure from specific shortcomings. Customers are hesitant to recommend and may highlight negative aspects when discussing the product or service.

- 2.6 to 3.5: \'Neutral\': Represents indifference; experiences are neither significantly positive nor negative. Recommendations may be lukewarm or noncommittal. Short feedback with no/minimal context

- 3.6 to 4.5: \'Satisfied\': Customers are generally pleased and feel their expectations have been met. They are likely to recommend the product or service, acknowledging some minor flaws.

- 4.6 to 5.0: \'Excited\': Shows high enthusiasm and exceeded expectations. Customers actively promote the product or service, sharing positive experiences and recommending it highly.5. Provide a detailed description of the sentiment.
6. Ensure the output is ONLY a valid JSON format with sentiment_score, sentiment_category, sentiment_description only
<OUTPUT Format>
{\"sentiment_score\": int, \"sentiment_category\":string, \"sentiment_description\":short description only}
</OUTPUT FORMAT>

</INSTRUCTIONS>
<EXAMPLE>

input: Why are you MORE satisfied with DBS\\\' local payment and transfer services than [Field-Competitor_Bank]\\\'s?: More ATMs, very easy to use (payment/transfer)
output: {
\\\"sentiment_score\\\": 3.7,
\\\"sentiment_category\\\": \\\"Satisfied\\\",
\\\"sentiment_description\\\": \\\"The customer expresses a higher satisfaction with DBS due to more ATMs and easier usability compared to [Field-Competitor_Bank].\\\"
}

input: Why are you LESS satisfied with DBS\\\' local payment and transfer services than [Field-Competitor_Bank]\\\'s?: Credit card rebate payment without tie to monthly spending (payment/transfer)
output: {
\\\"sentiment_score\\\": 2.4,
\\\"sentiment_category\\\": \\\"Unsatisfied\\\",
\\\"sentiment_description\\\": \\\"The customer is less satisfied due to the lack of tied rebates with DBS compared to [Field-Competitor_Bank], suggesting a desire for better benefits.\\\"
}

input: Uninstall paylah when after reinstall have difficulty authentication my email when I tried many times
output: {"sentiment_score\\\":1.3,\\\"sentiment_category":"Frustrated","sentiment_description":"The customer expresses significant frustration due to difficulties in re-authenticating their email after reinstalling the Paylah app, despite multiple attempts. This experience has likely led to a negative perception of the app\\\'s usability and customer support.\\\"}

input: Meh
output: {"sentiment_score":2.6,"sentiment_category":"Neutral","sentiment_description":"The customer has no comments and hence is neutral."}

</EXAMPLE>"""


    # Configure the model
    model = GenerativeModel(
        "gemini-1.0-pro-002",
        system_instruction=[system_instruction]
    )

    # Setup generation configuration
    generation_config = {
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1,
    }

    # Define safety settings
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [text],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    
    json_data= responses.text
    clean_json_string = json_data.strip('```json \n')
    data= json.loads(clean_json_string)

    # return json_data


    return (data["sentiment_score"], data["sentiment_category"])

text_input = """Why are you MORE satisfied with DBS Investment products than [Field-Competitor_Bank]'s Investment products?

(Please do not provide any sensitive personal information, including login passwords or one-time passwords.) Easy to trade """
response_output = classify_sentiment(text_input)
print(response_output)


(4.2, 'Satisfied')
